In [1]:
#bring imports here
import json
import requests
import numpy as np
import pandas as pd
import os
import sys
from time import sleep

## Scraping the Data

#### A big shoutout to Wesley Beckner for developing his package "salty" from which the majority of this code shown here came from. He has done some really cool research with ionic liquids, check out his github at https://github.com/wesleybeckner/salty to see his original code, and other cool features of his package.

While the NIST database is specifically directed towards ionic liquids, there is actually a lot of deep eutectic sovlent (DES) data as well. We can basically employ the same methods Wesley did to get this very useful data.

When you search for data on NIST, there are specific 4 letter tags for each property. You can find these tags and get the url for the data page by inspecting the webpage and going to the network dection. Perform the search, and the full url with the 4 letter tag will be shown. ILThermo's 4-letter tag for refractive index is bByM. I Performed a search for all refractive index data with keyword as 'Deep Eutectic Solvents' and returned a decent amount of data.

URL for the data: https://ilthermo.boulder.nist.gov/ILT2/ilsearch?cmp=&ncmp=0&year=&auth=&keyw=deep%20eutectic%20solvent&prp=bByM

A note to save the json files outside of a github repo, preferably a local folder on your device. I ran into permission issues when trying to save the scraped data directly into my github repo

In [2]:
paper_url = "https://ilthermo.boulder.nist.gov/ILT2/ilsearch?cmp=&ncmp=0&year=&auth=&keyw=Deep%20eutectic%20solvent&prp=bByM"
 
r = requests.get(paper_url)
header = r.json()['header']
papers = r.json()['res']
i = 1 
data_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilset?set={paper_id}'
for paper in papers[:]: 
    
    r = requests.get(data_url.format(paper_id=paper[0]))
    data = r.json()['data']
    with open("/Users/jaime/Desktop/nist_data/refractive_index/%s.json" % i, "w") as outfile: #set destination path for files
        json.dump(r.json(), outfile)
    #then do whatever you want to data like writing to a file
    sleep(0.5) #import step to avoid getting banned by server
    i += 1

## Cleaning up the data and adding to a pandas dataframe

The data has been scraped as 34 seperate json files. We can now use this code below to parse each file, extract the desired columns, and sort into a dataframe. A lot of the cleaning uses boolean logic because the data in the files are not all organized identically. I had to go in and see how they were organized to come up with some conditions that would parse accordingly based on the organization of the file. I frist developed this for one dataset (conductivity) and found it can be used with most of the others as well. There could be a little thing or two that may need adjusting based on what data is being scrapped, but in general it is mostly unviersal for the DES data on NIST

In [3]:
outer_old = pd.DataFrame()
outer_new = pd.DataFrame()

for i in range(34): #edit the number based on how many files were scraped
    with open("/Users/jaime/Desktop/nist_data/refractive_index/%s.json" % str(i+1)) as json_file:

        #grab data, data headers (names), the salt name
        json_full = json.load(json_file)                    
        json_data = pd.DataFrame(json_full['data'])
        json_datanames = np.array(json_full['dhead'])        # make names into array to add as columns headers for df
        json_data.columns =  json_datanames
        json_saltname = pd.DataFrame(json_full['components'])#components section contains names of DES components
        #print(json_saltname['name'])                #grabbing the HBD and HBA 

        inner_old = pd.DataFrame()
        inner_new = pd.DataFrame()

        #loop through the columns of the data, note that some of the 
        #json files are missing pressure data. 
        for indexer in range(len(json_data.columns)):        
            grab=json_data.columns[indexer]
            list = json_data[grab]
            my_list = [l[0] for l in list]
            dfmy_list = pd.DataFrame(my_list)
            dfmy_list.columns = [json_datanames[indexer][0]]
            inner_new = pd.concat([dfmy_list, inner_old], axis=1)
            inner_old = inner_new

        #print(inner_old.columns)


        #add the MW for HBD and HBA
        for i in range(len(json_saltname['name'])):
            if 'chloride' in json_saltname['name'][i] or 'bromide' in json_saltname['name'][i]:
                inner_old['HBA_MW']=json_saltname['mw'][i]
            else:
                inner_old['HBD_MW']=json_saltname['mw'][i]
                

        #add the DES components, i.e. HBA and HBD
        # they are not always listed in the same order on nist data, i.e., HBA always first. Will figure out later.
        for i in range(len(json_saltname['name'])):
            if 'chloride' in json_saltname['name'][i] or 'bromide' in json_saltname['name'][i]:
                inner_old['HBA']=json_saltname['name'][i]
            else:
                inner_old['HBD']=json_saltname['name'][i]
                
        #loop through the column names of the dataframe
        for j in range(len(inner_old.columns)):
            #if the words Mole fraction and a halogen are contained, values are correct and no value editing 
            #necessary and column is simply renamed to HBA mole fraction.
            if 'Mole fraction' in inner_old.columns[j] and 'chloride' in inner_old.columns[j] or 'Mole fraction' in inner_old.columns[j] and 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
            #if the words Mole Ratio and a halogen are contained, dataset was mislabeled but values are correct.
            #only need to rename column to HBA mole fraction.
            elif 'Mole ratio' in inner_old.columns[j] and 'chloride' in inner_old.columns[j] or 'Mole ratio' in inner_old.columns[j] and 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
            #if the words mole ratio are present, but no halogens, the ratio of the HBD is displayed and needs
            #to be changed to HBA mole fraction. First relabel the colum as HBA mole fraction.
            elif 'Mole ratio' in inner_old.columns[j] and not 'chloride' in inner_old.columns[j] or 'Mole ratio' in inner_old.columns[j] and not 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
                #apparently the numbers are strings so change to integer. May need to do this for every other column
                inner_old['HBA Mole Fraction'] = inner_old['HBA Mole Fraction'].astype(int)
                #next make an empty list that will hold all the new HBA mole fractions
                mole_fractions_list = []
                #loop through every HBD ratio in the column
                for k in range(len(inner_old['HBA Mole Fraction'])):
                    #Calculate the HBA mole fraction from every HBD ratio and append to the list
                    mole_fractions_list.append(1/(1+inner_old['HBA Mole Fraction'][k]))
                #finally make the list the new mole fraction column in the dataframe
                inner_old['HBA Mole Fraction'] = mole_fractions_list
            #in the last case, if the word mole fraction is present but not a halogen, HBD mole fraction is displayed.
            #Follow simialr process as before
            elif 'Mole fraction' in inner_old.columns[j] and not 'chloride' in inner_old.columns[j] or 'Mole fraction' in inner_old.columns[j] and not 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
                #convert to float instead since it is a decimal
                inner_old['HBA Mole Fraction'] = inner_old['HBA Mole Fraction'].astype(float)
                #empty list
                mole_fractions_list = []
                #loop through column
                for k in range(len(inner_old['HBA Mole Fraction'])):
                    #subtract 1 from HBD mole fraction to get HBA mole fraction and append to list
                    mole_fractions_list.append(1 - inner_old['HBA Mole Fraction'][k])
                #replace column   
                inner_old['HBA Mole Fraction'] = mole_fractions_list


        #add to the growing dataframe
        outer_new = pd.concat([inner_old, outer_old], axis = 0, ignore_index = True)
        outer_old = outer_new
    
outer_old.head(50)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,HBA,HBA Mole Fraction,HBA_MW,HBD,HBD_MW,"Pressure, kPa",Refractive index (Na D-line),"Temperature, K"
0,tetrapropylammonium bromide,0.333,266.27,"1,6-hexanediol",118.18,101.3,1.47923,313.15
1,NaN,0.5,NaN,urea,60.06,99.6,1.493,293.15
2,benzyltriphenylphosphonium chloride,0.833,388.87,glycerol,92.09,101.3,1.5502,328.15
3,benzyltriphenylphosphonium chloride,0.833,388.87,glycerol,92.09,101.3,1.5168,338.15
4,benzyltriphenylphosphonium chloride,0.75,388.87,"1,2-ethanediol",62.07,101.3,1.5258,328.15
5,benzyltriphenylphosphonium chloride,0.75,388.87,"1,2-ethanediol",62.07,101.3,1.5309,338.15
6,methyltriphenylphosphonium bromide,0.16,357.23,"1,2-ethanediol",62.07,101,1.5398,298.15
7,methyltriphenylphosphonium bromide,0.2,357.23,"1,2-ethanediol",62.07,101,1.553,298.15
8,methyltriphenylphosphonium bromide,0.25,357.23,"1,2-ethanediol",62.07,101,1.5678,298.15
9,methyltriphenylphosphonium bromide,0.2,357.23,glycerol,92.09,101,1.5597,298.15


It appears there are some Nan's for the HBA column. I looked at the original data files and this was a weird ionic liquid that used urea, so technically not a DES which is why the parsing in my code didn't pick it up and name it accordingly. Just going to drop those rows and the rest should be fine, also will save this dataframe as a csv.

In [5]:
outer_old.dropna(inplace = True)

In [6]:
pd.DataFrame.to_csv(outer_old, path_or_buf='ref_index.csv', index=False)

In [7]:
pd.read_csv('ref_index.csv')

,HBA,HBA Mole Fraction,HBA_MW,HBD,HBD_MW,"Pressure, kPa",Refractive index (Na D-line),"Temperature, K"
0,tetrapropylammonium bromide,0.3330,266.27,"1,6-hexanediol",118.18,101.3,1.47923,313.15
1,benzyltriphenylphosphonium chloride,0.8330,388.87,glycerol,92.09,101.3,1.55020,328.15
2,benzyltriphenylphosphonium chloride,0.8330,388.87,glycerol,92.09,101.3,1.51680,338.15
3,benzyltriphenylphosphonium chloride,0.7500,388.87,"1,2-ethanediol",62.07,101.3,1.52580,328.15
4,benzyltriphenylphosphonium chloride,0.7500,388.87,"1,2-ethanediol",62.07,101.3,1.53090,338.15
5,methyltriphenylphosphonium bromide,0.1600,357.23,"1,2-ethanediol",62.07,101.0,1.53980,298.15
6,methyltriphenylphosphonium bromide,0.2000,357.23,"1,2-ethanediol",62.07,101.0,1.55300,298.15
7,methyltriphenylphosphonium bromide,0.2500,357.23,"1,2-ethanediol",62.07,101.0,1.56780,298.15
8,methyltriphenylphosphonium bromide,0.2000,357.23,glycerol,92.09,101.0,1.55970,298.15
9,methyltriphenylphosphonium bromide,0.2500,357.23,glycerol,92.09,101.0,1.57150,298.15
